In [1]:
!pip install transformers
!pip install tensorflow-gpu
!pip install tf-keras
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        

2024-05-03 20:43:11.117839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

eos_token = tokenizer.sep_token_id
pad_token = tokenizer.pad_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-03 20:43:13.710896: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:13.739773: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:13.739807: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:

In [3]:
input_sequences = []
target_tokens = []
max_length = 64

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')

        # Encode the texts. Adding the EOS token is important for GPT to know when a sentence ends
        question_tokens = tokenizer.encode(question, add_special_tokens=False)
        answer_tokens = tokenizer.encode(answer, add_special_tokens=False) + [eos_token] # Ensure EOS token is added to the answer

        # Construct sequences of max_length or shorter
        for i in range(0, len(answer_tokens)):
            # Input sequence: question + end token + part of the answer
            seq = question_tokens + [eos_token] + answer_tokens[:i]
            if len(seq) > max_length:
                break

            # Ensure sequences are padded to have uniform length
            input_sequences.append(seq + [pad_token] * (max_length - len(seq)))  # Padding

            # Target token is the next token in the answer
            target_tokens.append(answer_tokens[i])

In [4]:
for seq in input_sequences[:10]:
    #print(seq)
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))
    print('\n')
          
for seq in target_tokens[:10]:
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))

hi, how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [5]:
input_sequences = tf.constant(input_sequences, dtype=tf.int32)
target_tokens = tf.constant(target_tokens, dtype=tf.int32)
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_tokens))
dataset = dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

vocab_size = tokenizer.vocab_size

In [6]:
# Create the model

In [7]:
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
outputs = bert_model(input_ids)[0]  # We use only the last hidden state
cls_output = outputs[:, -1, :]  # We use the output of the last token for prediction
predictions = tf.keras.layers.Dense(vocab_size, activation='softmax')(cls_output)

model = tf.keras.Model(inputs=input_ids, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 64)]              0         
                                                                 
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPo   109482240 
 )                           olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 64, 768),                        
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                             

In [8]:
if tf.test.gpu_device_name():
    print(f"Default GPU Device: {tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


2024-05-03 20:43:28.437812: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:28.437904: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:28.437924: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:28.438148: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 20:43:28.438159: I tensorflow/core/common_runtime/gpu/gpu

In [9]:
epochs = 25

# Train the model
history = model.fit(dataset, epochs=epochs)

Epoch 1/25


I0000 00:00:1714769045.391304    3845 service.cc:145] XLA service 0x7fb54020b130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714769045.391370    3845 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-05-03 20:44:05.400231: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 20:44:05.420987: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1714769045.469351    3845 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1139/1139 [==============================] - 222s 156ms/step - loss: 5.7263
Epoch 2/25
1139/1139 [==============================] - 171s 150ms/step - loss: 5.2231
Epoch 3/25
1139/1139 [==============================] - 165s 145ms/step - loss: 5.1820
Epoch 4/25
1139/1139 [==============================] - 165s 145ms/step - loss: 5.1605
Epoch 5/25
1139/1139 [==============================] - 168s 148ms/step - loss: 5.1480
Epoch 6/25
1139/1139 [==============================] - 167s 147ms/step - loss: 5.1382
Epoch 7/25
1139/1139 [==============================] - 173s 152ms/step - loss: 5.1341
Epoch 8/25
1139/1139 [==============================] - 173s 152ms/step - loss: 5.1239
Epoch 9/25
1139/1139 [==============================] - 165s 145ms/step - loss: 5.1142
Epoch 10/25
1139/1139 [==============================] - 172s 151ms/step - loss: 5.1127
Epoch 11/25
1139/1139 [==============================] - 170s 150ms/step - loss: 5.1104
Epoch 12/25
1139/1139 [=============================

In [10]:
model_save_path = "my_transformer_model"
tf.saved_model.save(model, model_save_path)

INFO:tensorflow:Assets written to: my_transformer_model/assets


INFO:tensorflow:Assets written to: my_transformer_model/assets


In [11]:
from tensorflow.keras.models import load_model
from transformers import BertTokenizer, TFBertModel
eos_token = tokenizer.sep_token_id

model_save_path = "my_transformer_model"
loaded_model = tf.saved_model.load(model_save_path)

In [32]:
def generate_until_sep_token(starting_text, max_tokens=50):
    text = starting_text
    for _ in range(max_tokens):
        token_ids = tokenizer.encode(text, add_special_tokens=False) + [eos_token]
        
        # Pad token_ids to ensure the input length is exactly 64
        if len(token_ids) < 64:
            token_ids += [pad_token] * (64 - len(token_ids))
        elif len(token_ids) > 64:
            token_ids = token_ids[-64:]  # Truncate to the last 64 tokens if exceeding the length
        
        input_sequence = tf.constant([token_ids], dtype=tf.int32)

        # Using the serving function with named arguments as expected by the model
        serving_fn = loaded_model.signatures['serving_default']
        predictions = serving_fn(input_ids=input_sequence)

        # Accessing the prediction using the 'dense' key
        predicted_token_id = tf.argmax(predictions['dense'], axis=-1).numpy()[0]

        #if predicted_token_id == eos_token:
            #break
        
        # Decode the predicted token ID to the corresponding token
        predicted_token = tokenizer.decode([predicted_token_id])
        text += predicted_token

    return text


In [33]:
starting_text = "hello, how are you?"
generated_text = generate_until_sep_token(starting_text, max_tokens=50)
print("Generated text:", generated_text)

Generated text: hello, how are you?[SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP][SEP]
